In [4]:
import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd

fishing_hours_path = '/Users/loucas/Documents/ORG/things too big for git/fishing_hours.csv'
trawling_path = '/Users/loucas/Documents/ORG/things too big for git/surface_subsurface.csv'
squares_path = '/Users/loucas/Documents/ORG/things too big for git/squares.geojson'

fishing_hours = pd.read_csv(fishing_hours_path)
trawling = pd.read_csv(trawling_path)
squares = gpd.read_file(squares_path)

In [5]:
def check_column_values(df1, df2, column_name):
    # Check if the column exists in both DataFrames
    if column_name not in df1.columns or column_name not in df2.columns:
        return None, f"Column '{column_name}' not found in one or both DataFrames"
    
    # Get the unique values from both DataFrames
    df1_values = set(df1[column_name].unique())
    df2_values = set(df2[column_name].unique())
    
    # Find values in df1 that are not in df2
    missing_values = list(df1_values - df2_values)
    print(len(missing_values))


check_column_values(squares, fishing_hours, 'c_squar')

84


In [6]:

def add_column_from_df2_to_df1(df1, df2, match_column, new_column):
    # Check if the columns exist in the respective DataFrames
    if match_column not in df1.columns or match_column not in df2.columns:
        return None, f"Match column '{match_column}' not found in one or both DataFrames"
    if new_column not in df2.columns:
        return None, f"New column '{new_column}' not found in the second DataFrame"
    
    # Perform a left merge
    merged_df = df1.merge(df2[[match_column, new_column]], 
                          on=match_column, 
                          how='left')
    
    # Check for any rows that didn't find a match
    unmatched = merged_df[merged_df[new_column].isna()]
    
    if not unmatched.empty:
        print(f"Warning: {len(unmatched)} rows in df1 did not find a match in df2.")
    
    return merged_df

merged = add_column_from_df2_to_df1(squares, fishing_hours, 'c_squar', 'mw_fshn')
merged = add_column_from_df2_to_df1(merged, trawling, 'c_squar', 'sbsrfc_sr')
merged = add_column_from_df2_to_df1(merged, trawling, 'c_squar', 'srfc_sr')


In [7]:
# Convert to epsg 3035
merged = merged.to_crs(epsg=3035)

In [8]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import numpy as np

def gdf_to_raster(gdf, column, output_path, resolution=None):
    """
    Convert a GeoDataFrame with polygon grid cells to a raster.
    
    :param gdf: GeoDataFrame with polygon geometries representing a grid
    :param column: Name of the column to use for raster values
    :param output_path: Path to save the output raster file
    :param resolution: Resolution of the output raster. If None, it's calculated from the data
    :return: None
    """
    # Ensure the GeoDataFrame is in a projected CRS
    if gdf.crs is None or gdf.crs.is_geographic:
        raise ValueError("GeoDataFrame must be in a projected CRS")

    # Get the extent of the grid
    minx, miny, maxx, maxy = gdf.total_bounds

    # Calculate resolution if not provided
    if resolution is None:
        # Assumes square cells and takes the width of the first geometry
        resolution = gdf.geometry.iloc[0].bounds[2] - gdf.geometry.iloc[0].bounds[0]

    # Calculate raster dimensions
    width = int((maxx - minx) / resolution)
    height = int((maxy - miny) / resolution)

    # Create the transform
    transform = from_origin(minx, maxy, resolution, resolution)

    # Rasterize the polygons
    shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf[column]))
    raster = rasterize(shapes=shapes, out_shape=(height, width), transform=transform, fill=0, all_touched=True)

    # Write the raster
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=raster.dtype,
        crs=gdf.crs,
        transform=transform,
    ) as dst:
        dst.write(raster, 1)

    print(f"Raster saved to {output_path}")

gdf_to_raster(merged, 'sbsrfc_sr', '/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/fishing_intensity/subsurface_swept_ratio.tif')
gdf_to_raster(merged, 'srfc_sr', '/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/fishing_intensity/surface_swept_ratio.tif')
gdf_to_raster(merged, 'mw_fshn', '/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/fishing_intensity/fishing_hours.tif')

Raster saved to /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/fishing_intensity/subsurface_swept_ratio.tif
Raster saved to /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/fishing_intensity/surface_swept_ratio.tif
Raster saved to /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/fishing_intensity/fishing_hours.tif
